In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
  This script generates verification maps:
      Inputs:
          Forecast vector file (geojson format)
          Predictand data covering the training period
      Outputs:
          Verification netCDF file

@author: thembani - original version
@author: pwolski - revised to the level that hardly anything of the original is left currently
         Nov 2022 - using xarray and allow more general format of netcdf files, also some changes to UI
         Aug 2023 - implemented threading and CSV format of csv files
"""

import os, sys, time
from datetime import datetime, timedelta
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import geojson, json

import matplotlib
matplotlib.use('agg')
from pathlib import Path
import matplotlib.pyplot as plt

from PyQt5 import QtCore, QtGui, QtWidgets, uic
from PyQt5.QtCore import QThread, QObject, QDate, QTime, QDateTime, Qt

from PyQt5.QtCore import pyqtSignal

import warnings
warnings.filterwarnings("ignore")

#rioxarray has to be installed, but does not have to be loaded
import cftime
import xarray as xr
#from geocube.api.core import make_geocube
import geopandas as gpd
import matplotlib.colors as colors
import cartopy.crs as ccrs

#defining fixed things
version="1.0"
qtCreatorFile = "forecast_synthesis.ui"
settingsfile = 'forecast_synthesis.json'

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# these are parameters for selection of the season over which forecast is to be evaluated. 
# First value is duration of the period, second is the index of the LAST month of the target period.
# This index is 1-based, i.e. Jan is 1, Feb is 2 etc.
# For example, for JFM, the first value will be 3, second will be 3 as March is the last month of JFM.
seasonParam = {
           'JFM':[3,3],
           'FMA':[3,4],
           'MAM':[3,5],
           'AMJ':[3,6],
           'MJJ':[3,7],
           'JJA':[3,8],
           'JAS':[3,9],
           'ASO':[3,10],
           'SON':[3,11],
           'OND':[3,12],
           'NDJ':[3,1],
           'DJF':[3,2]
            }

signalAgreeLevels={"",
                   "low",
                   "moderate",
                   "high"}

skillLevels={"",
             "all low",
             "mixed/moderate",
             "all high"}

fcstCategories={"":0,
                "below normal":1,
                "normal to below":2,
                "normal to above":3,
                "above normal":4}

#first key is skillLevel,second is signalAgreeLevel
confLevels={
    "":{"":"",
        "low":"",
        "moderate":"",
        "high":""},
    "all low":{"":"",
        "low":"low",
        "moderate":"low",
        "high":"moderate"},
    "mixed/moderate":{"":"",
        "low":"low",
        "moderate":"moderate",
        "high":"high"},
    "all high":{"":"",
        "low":"moderate",
        "moderate":"high",
        "high":"very high"},
}


msgColors={"ERROR": "red",
           "INFO":"blue",
           "RUNTIME":"grey",
           "NONCRITICAL":"red",
           "SUCCESS":"green"
          }



# program flow functions
################################################################################################################

def openHelp():
    webbrowser.open(helpfile)

def clearLog():
    global window
    window.logWindow.clear()
        
def closeApp():
    sys.exit(app.exec_())

def addzonesFile():
    showMessage("Selecting forecast vector file...")
    global config
    fileName = QtWidgets.QFileDialog.getOpenFileName(window,
              'Select File', '..' + os.sep, filter="GeoJson File (*.geojson)")
    if fileName[0]!="":
        #widget will return empty string if selection cancelled        
        showMessage("Checking {}...".format(fileName[0]))
        try:
            with open(fileName[0]) as f:
                jsonfile = geojson.load(f)
                #at this stage, just keys of properties are needed. these can be read from the first feature only
                feature = jsonfile['features'][0]
                variables=list(feature["properties"].keys())
            
        except:
            showMessage("Could not read {}, check if it is a valid geojson file".format(fileName[0]), "ERROR")
            return
        
        if len(variables)==0:
            showMessage("Forecast geojson file should have at least one property associated with geometric features. Current file has 0. Please check if you loaded correct file", "ERROR")
            return
        
        showMessage("Zones will be read from: {}".format(fileName[0]), "INFO")
        
        window.zonesFilePath.setText(fileName[0])
        config['zonesFile'] = {"file": '', "ID": 0, "variable": []}
        config['zonesFile']['file'] = fileName[0]
        
        window.zonesFileVariable.clear()
        for variable in variables:
            window.zonesFileVariable.addItem(variable)
            config['zonesFile']['variable'].append(variable)
            
        config['zonesFilereference']=jsonfile
        config['zonesVariable']=variable
        window.zonesFileVariable.setCurrentText(variable)
        resetAll()
        window.saveButton.setEnabled(True)
#            print(config)
                        
    else:
        showMessage("Selecting forecast file aborted")
        

def saveZoneData():
    #writing zone data to config
    zone=window.zoneCode.currentText()
    print("Zone",zone)
    zoneData={}
    
    signalAgree=window.signalAgree.currentText()
    skillLevel=window.skillLevel.currentText()
    
    zoneData["signalAgree"]=signalAgree
    zoneData["skillLevel"]=skillLevel
    zoneData["fcstCategory"]=window.fcstCategory.currentText()
    
    confLevel=confLevels[skillLevel][signalAgree]
    zoneData["confLevel"]=confLevel
    window.confLevel.setText(confLevel)

    config['zoneData'][zone]=zoneData
    
    
    #print current status of all zones/variables in log window
    clearLog()
    showMessage("Current values:", "INFO")
    for zone in config['zoneData'].keys():
        zoneData=config['zoneData'][zone]
        showMessage("\nZone {}\n Skill level:{}\n Signal agreement:{}\n Confidence: {}\n Forecast category: {}".format(zone,config['zoneData'][zone]['skillLevel'],config['zoneData'][zone]['signalAgree'],config['zoneData'][zone]['confLevel'],config['zoneData'][zone]['fcstCategory']), "INFO")

    
def loadZoneData():
    #loading zone data from config to UI
    cont=True
    zone=window.zoneCode.currentText()
    print("Zone",zone)
    zoneData=config['zoneData'][zone]
    window.signalAgree.setCurrentText(zoneData["signalAgree"])
    window.skillLevel.setCurrentText(zoneData["skillLevel"])
    window.fcstCategory.setCurrentText(zoneData["fcstCategory"])
    window.confLevel.setText(zoneData["confLevel"])


def resetAll():
    #this is on change of geojson file
    #resetting zone data
    setConfigDefaults()
    resetZones()
    resetZoneData()

def resetZoneData():
    #resetting zone data
    cont=True
    zone=window.zoneCode.currentText()
    print("Zone",zone)
    zoneData=config['zoneData'][zone]
    window.signalAgree.setCurrentText(zoneData["signalAgree"])
    window.skillLevel.setCurrentText(zoneData["skillLevel"])
    window.fcstCategory.setCurrentText(zoneData["fcstCategory"])
    window.confLevel.setText(zoneData["confLevel"])


def resetZones():
    window.zoneCode.clear()
    jsonfile=config['zonesFilereference']
    zonesVariable=window.zonesFileVariable.currentText()
    config['zonesVariable']=zonesVariable
    config["zoneData"]={}
    codes=[]
    for feature in jsonfile['features']:
        print(feature["properties"])
        code=str(feature["properties"][zonesVariable])
        print("code",code)
        if code in codes:
            showMessage("Zone names should be unique. Check if you selected correct variable storing zone ID", "NONCRITICAL")            
        else:
            codes.append(code)
        window.zoneCode.addItem(code)
        zoneData={"signalAgree": '', "skillLevel": '',"fcstCategory":'',"confLevel":''}
        config['zoneData'][code]=zoneData
        print("data",config['zoneData'])
    loadZoneData()
    
    
    
def writeOutput():
    cont=True
    #checking if all populated
    for zone in config['zoneData'].keys():
        for var in config['zoneData'][zone].keys():
            if config['zoneData'][zone][var]=="":
                showMessage("{} for zone {} missing".format(var,zone), "ERROR")
                return
            
    if config['outDir']=="":
        showMessage("output directory not set", "ERROR")
        return
    
    if window.fcstYear.text()=="":
        showMessage("forecast year not set", "ERROR")
        return
    else:
        fcstYear=window.fcstYear.text()
        config['fcstYear']=fcstYear
        
    fcstPeriod=window.fcstPeriod.currentText()
        
        
    showMessage("all information provided. Writing output...", "INFO")
    geojsondict0=config['zonesFilereference']
    variable=config['zonesVariable']
    
    #need to do this to drop superfluous variables from dictionary
    geojsondict=json.loads(json.dumps(geojsondict0))
    
    #removing old columns, keeping only zone ID one
    for i,feature in enumerate(geojsondict0['features']):
        zone=str(geojsondict0['features'][i]["properties"][variable])
        for key in geojsondict0['features'][i]["properties"]:
            if key!=variable:
                geojsondict['features'][i]["properties"].pop(key)
                
    #adding new entries
    for i,feature in enumerate(geojsondict['features']):
        zone=str(geojsondict['features'][i]["properties"][variable])
        fcstCategory=config['zoneData'][zone]["fcstCategory"]
        confLevel=config['zoneData'][zone]["confLevel"]
        fcstCategoryCode=fcstCategories[fcstCategory]
        skillLevel=config['zoneData'][zone]["skillLevel"]
        signalAgree=config['zoneData'][zone]["signalAgree"]
        
        geojsondict['features'][i]["properties"]["finalcode_{}-{}".format(fcstYear,fcstPeriod)]=fcstCategoryCode    
        geojsondict['features'][i]["properties"]["finalcategory_{}-{}".format(fcstYear,fcstPeriod)]=fcstCategory    
        geojsondict['features'][i]["properties"]["finalconfidence_{}-{}".format(fcstYear,fcstPeriod)]=confLevel
        geojsondict['features'][i]["properties"]["agreement_{}-{}".format(fcstYear,fcstPeriod)]=signalAgree
        geojsondict['features'][i]["properties"]["skill_{}-{}".format(fcstYear,fcstPeriod)]=skillLevel
            
    outputfile="{}/forecast_{}-{}.geojson".format(config['outDir'],config['fcstYear'],window.fcstPeriod.currentText())
    
    if os.path.exists(outputfile):
        showMessage("outputfile {} exists. Overwriting...".format(outputfile), "INFO")
        
    with open(outputfile,'w') as f:
        json.dump(geojsondict, f)

    showMessage("written {}".format(outputfile), "RUNTIME")
    
    #need to finish this one still
    #plotforecast(geojsondict)

        
def plotforecast(_geojsondict):
    cont=True
    #to be done    
    
    
def getOutDir():
    global config
    showMessage("Setting output directory...")
    outDir=QtWidgets.QFileDialog.getExistingDirectory(directory='..' + os.sep)
    if outDir!='':
        config['outDir']=outDir 
        window.outDirPath.setText(outDir)
        showMessage("Output will be written to {}".format(outDir), "INFO")
    else:
        showMessage("Selecting output directory aborted")

    
def setConfigDefaults():
    config = {}
    
    #output directory
    config['outDir'] = ''   

    #zones file
    config['zonesFile'] = {"file": '',"variable": [], "ID": None}

    config['fcstYear'] = ""
    config['fcstPeriod'] = {"season": list(seasonParam.keys()),
                        "indx": 2}
    
    config['signalAgree'] = {"level": list(signalAgreeLevels),
                        "indx": 0}
    
    config['skillLevel'] = {"level": list(skillLevels),
                        "indx": 0}

    config['fcstCategory'] = {"category": list(fcstCategories.keys()),
                        "indx": 0}

    config['zoneData'] = {}
    config['zonesFilereference'] = ''
    config['zonesVariable'] = ''
    
    return config    



        
def populateUI():
    showMessage("Populating UI...")
    #this populates UI based on values in config dictionary
    global window

    #output directory
    window.outDirPath.setText(config.get('outDir'))
    
    #zones file
    window.zonesFilePath.setText(os.path.basename(config.get('zonesFile').get('file')))
    for var in config.get('zonesFile').get('variable'):
        window.zonesFileVariable.addItem(var)
    if type(config.get('zonesFile').get('ID')) == type(0): 
        window.zonesFileVariable.setCurrentIndex(config.get('zonesFile').get('ID'))

    #forecast
    periodxs = config.get('fcstPeriod').get('season')
    for periodx in periodxs:
        window.fcstPeriod.addItem(periodx)  
    if type(config.get('fcstPeriod').get('indx')) == type(0): 
        window.fcstPeriod.setCurrentIndex(config.get('fcstPeriod').get('indx'))
    window.fcstYear.setText(str(config.get('fcstYear')))
    
    levels = config.get('signalAgree').get('level')
    for levelx in levels:
        window.signalAgree.addItem(levelx)  
    if type(config.get('signalAgree').get('indx')) == type(0): 
        window.signalAgree.setCurrentIndex(config.get('signalAgree').get('indx'))

    levels = config.get('skillLevel').get('level')
    for levelx in levels:
        window.skillLevel.addItem(levelx)  
    if type(config.get('skillLevel').get('indx')) == type(0): 
        window.skillLevel.setCurrentIndex(config.get('signalAgree').get('indx'))

        
    levels = config.get('fcstCategory').get('category')
    for levelx in levels:
        window.fcstCategory.addItem(levelx) 
    if type(config.get('fcstCategory').get('indx')) == type(0): 
        window.fcstCategory.setCurrentIndex(config.get('fcstCategory').get('indx'))
        
    ## attaching signals
    #it is obvious what these do
    window.outDirButton.clicked.connect(getOutDir)
    window.zonesFileButton.clicked.connect(addzonesFile)

    #again, obvious actions
    window.zonesFileVariable.textActivated.connect(resetZones)
    window.zoneCode.textActivated.connect(loadZoneData)
    window.signalAgree.textActivated.connect(saveZoneData)
    window.skillLevel.textActivated.connect(saveZoneData)
    window.fcstCategory.textActivated.connect(saveZoneData)    
    window.exitButton.clicked.connect(closeApp)
    window.helpButton.clicked.connect(openHelp)
    window.clearLogButton.clicked.connect(clearLog)
    window.saveButton.clicked.connect(writeOutput)
    window.saveButton.setEnabled(False)
    showMessage("UI ready", "INFO")



    

def showMessage(_message, _type="RUNTIME"):
    #this print messages to log window, which are generated outside of the threaded function
    global window
    _color=msgColors[_type]
    _message = "<pre><font color={}>{}</font></pre>".format(_color, _message)
    window.logWindow.appendHtml(_message)
#    window.logWindow.update()
    window.logWindow.ensureCursorVisible()


    
    


#reading UI - has to be done before UI class is implemented
################################################################################################################

Ui_MainWindow, QtBaseClass = uic.loadUiType(qtCreatorFile)
    
    
class MyApp(QtWidgets.QMainWindow, Ui_MainWindow):
    def __init__(self):
        QtWidgets.QMainWindow.__init__(self)
        Ui_MainWindow.__init__(self)
        self.setupUi(self)
        
        
    def reportProgress(self, _tuple):
        #this print messages to log window, which are generated in the threaded function
        global window
        _message=_tuple[0]
        _type=_tuple[1]
        _color=msgColors[_type]
        if _type in ["ERROR","NONCRITICAL"]:
            _message="{}: {}".format(_type,_message)
        _message = "<pre><font color={}>{}</font></pre>".format(_color, _message)
        window.logWindow.appendHtml(_message)
        window.logWindow.ensureCursorVisible()
            


# this is where magic happens
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = MyApp()
    window.show()
    
    showMessage("Loading config...")
    config=setConfigDefaults()
    showMessage("Default settings loaded.", "INFO")
                

    # --- Load values from config file into the UI ---
    populateUI()
    window.confLevel.setStyleSheet("background-color:lightyellow")
    

    sys.exit(app.exec_())


# In[ ]:






# In[ ]:






{'SECONDARY': '2', 'PRIM_ID': 2, 'ID': 2, 'class4': 13, 'class3': 3, 'class2': 12, 'class1': 14, 'wavg': 2.3647209203238178, 'fcst_class': 2, 'avgHS': 55.88095238095238}
code 55.88095238095238
data {'55.88095238095238': {'signalAgree': '', 'skillLevel': '', 'fcstCategory': '', 'confLevel': ''}}
{'SECONDARY': '1', 'PRIM_ID': 1, 'ID': 1, 'class4': 7, 'class3': 4, 'class2': 11, 'class1': 9, 'wavg': 2.2989520132377277, 'fcst_class': 2, 'avgHS': 58.483870967741936}
code 58.483870967741936
data {'55.88095238095238': {'signalAgree': '', 'skillLevel': '', 'fcstCategory': '', 'confLevel': ''}, '58.483870967741936': {'signalAgree': '', 'skillLevel': '', 'fcstCategory': '', 'confLevel': ''}}
Zone 55.88095238095238
Zone 55.88095238095238
{'SECONDARY': '2', 'PRIM_ID': 2, 'ID': 2, 'class4': 13, 'class3': 3, 'class2': 12, 'class1': 14, 'wavg': 2.3647209203238178, 'fcst_class': 2, 'avgHS': 55.88095238095238}
code 2
data {'2': {'signalAgree': '', 'skillLevel': '', 'fcstCategory': '', 'confLevel': ''}}


In [175]:
import geojson, json
from matplotlib import pyplot as plt
import geopandas as gpd
import matplotlib.colors as colors
import cartopy.crs as ccrs
fcstFile="../test/tanzania/forecast_2023-OND.geojson"
with open(fcstFile) as f:
    geojsondict=geojson.load(f)
poly = gpd.read_file(fcstFile)
poly.loc[3,'finalcode_2023-OND']=3
poly

,Zone,finalcode_2023-OND,finalcategory_2023-OND,finalconfidence,agreement_2023-OND,skill_2023-OND,geometry
0,1,1,below normal,moderate,high,all low,"POLYGON ((37.57292 -11.62301, 37.57109 -11.622..."
1,2,2,normal to below,low,low,mixed/moderate,"POLYGON ((31.27085 -0.99725, 31.27092 -0.99725..."
2,3,3,normal to above,high,moderate,all high,"MULTIPOLYGON (((39.23492 -7.82301, 39.23506 -7..."
3,4,3,above normal,very high,high,all high,"MULTIPOLYGON (((39.07509 -6.53288, 39.07534 -6..."


In [182]:
gpd.GeoDataFrame(geojsondict['features'][0]).set_geometry('geometry')
#geojsondict['features'][0]

TypeError: Input must be valid geometry objects: Polygon

#/   - diagonal hatching
#\   - back diagonal
#|   - vertical
#-   - horizontal
#+   - crossed
#x   - crossed diagonal
#o   - small circle
#O   - large circle
#.   - dots
#*   - stars

In [184]:
hatchdict={"low":"",
          "moderate":"/",
          "high":"//",
          "very high":"///"}

cols={"below normal":'#d2b48c',
      "normal to below":'yellow',
      "normal to above":'#0bfffb',
      "above normal":'blue'}
hatchdict={"low":"",
          "moderate":".",
          "high":"..",
          "very high":"..."}

In [1]:
from matplotlib.patches import Patch
edgecolor="0.7"

legend_conf=[]
fig=plt.figure(figsize=(7,5))
pl=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
for conf in hatchdict.keys():
    print(conf)
    legend_entry=Patch(facecolor='white', edgecolor=edgecolor, hatch=hatchdict[conf],
                         label=conf)
    legend_conf.append(legend_entry)
    
    selpoly=poly[poly["finalconfidence"]==conf]
    if selpoly.shape[0]>0:
        selpoly.plot(column=code,hatch=hatchdict[conf], ax=pl, cmap=cmap, vmin=1, vmax=5, edgecolor="0.6",lw=0.5)
        #selpoly.boundary.plot(ax=pl, color="", lw=0.5)

        
legend_conf = []
for conf in hatchdict.keys():
    legend_entry=Patch(facecolor='white', edgecolor=edgecolor, hatch=hatchdict[conf],label=conf)
    legend_conf.append(legend_entry)

    
legend_cat = []
for x in cols.keys():
    legend_entry=Patch(facecolor=cols[x], edgecolor=edgecolor, label=x)
    legend_cat.append(legend_entry)
    

legend1=pl.legend(handles=legend_conf, loc=(1.01,0), title="Forecast confidence",handleheight=2, handlelength=3)
legend2=pl.legend(handles=legend_cat, loc=(1.01,0.5), title="Forecast category",handleheight=2, handlelength=3)
plt.gca().add_artist(legend1)

plt.subplots_adjust(right=0.7)
#figname="test.jpg"
#figname="{}/forecast_{}-{}.jpg".format(config['outDir'],config['fcstYear'],window.fcstPeriod.currentText())

#plt.savefig(figname)

NameError: name 'plt' is not defined

In [6]:
variable="Zone"
geojsondict=json.loads(json.dumps(geojsondict0))

for i,feature in enumerate(geojsondict0['features']):
        zone=str(geojsondict0['features'][i].properties[variable])
        for key in geojsondict0['features'][i].properties:
            if key!=variable:
                geojsondict['features'][i]["properties"].pop(key)
